In [ ]:
#=================================================Data Generator==========================================================
import os
import numpy as np
from  scipy import ndimage
import nibabel as nib
epoch_num = 40
kepoch_num = 20
batch_size = 4
tbatch_size = 4
spath = 'save_path'
os.mkdir(spath)
train_image_path = 'data_path'
test_image_path = 'test_data_path'

label = []
path = []
tlabel = []
tpath = []

k = os.listdir(train_image_path)
for j in k:
    path.append(train_image_path + '/' + j)   
    label.append(train_image_path + '/' + j)
k = os.listdir(test_image_path)
for j in k:
    path.append(test_image_path + '/' + j)   
    label.append(test_image_path + '/' + j)
    
steps_per_epoch = int(len(path)/batch_size)
tsteps_per_epoch = int(len(tpath)/tbatch_size)

def process(data_dir, label):
    if '_' in data_dir:
        img = nib.load(data_dir + '/T1W+C.nii').get_fdata()
        img2 = nib.load(data_dir + '/T2W.nii').get_fdata()
    else:
        img = nib.load(data_dir + '/T1W+C.nii').get_fdata()
        img2 = nib.load(data_dir + '/T2W.nii').get_fdata()
    lb = nib.load(label + '/TV_MASK.nii').get_fdata()
    
    img[np.where(np.isnan(img))] = 0
    img2[np.where(np.isnan(img2))] = 0
    lb[np.where(np.isnan(lb))] = 0    
    
    #=======augmentation=========#
    
    #flip_ratio = np.random.rand(1)
    #if flip_ratio >= 0.5:
        #img = np.fliplr(img)
        #img2 = np.fliplr(img2)
        #lb = np.fliplr(lb)
    #rot_angle = np.random.randint(-30, 30)
    #img = ndimage.rotate(img, rot_angle, reshape=False)
    #img2 = ndimage.rotate(img2, rot_angle, reshape=False)
    #lb = ndimage.rotate(lb, rot_angle, reshape=False)
    
    #=======augmentation=========#
    
    img = (img-np.mean(img))/(np.std(img))
    img2 = (img2-np.mean(img2))/(np.std(img2))
    
    img = np.expand_dims(img,-1)
    img2 = np.expand_dims(img2,-1)
    lb = np.expand_dims(lb,-1)  
    imgn = np.concatenate((img,img2),-1)
    return np.array(imgn,dtype=np.float32), np.array(lb,dtype=np.float32)
def tprocess(data_dir, label):
    if '_' in data_dir:
        img = nib.load(data_dir + '/T1W+C.nii').get_fdata()
        img2 = nib.load(data_dir + '/T2W.nii').get_fdata()
    else:
        img = nib.load(data_dir + '/T1W+C.nii').get_fdata()
        img2 = nib.load(data_dir + '/T2W.nii').get_fdata()
    lb = nib.load(label + '/TV_MASK.nii').get_fdata()
    
    img[np.where(np.isnan(img))] = 0
    img2[np.where(np.isnan(img2))] = 0
    lb[np.where(np.isnan(lb))] = 0    
    
    img = (img-np.mean(img))/(np.std(img))
    img2 = (img2-np.mean(img2))/(np.std(img2))
    
    img = np.expand_dims(img,-1)
    img2 = np.expand_dims(img2,-1)
    lb = np.expand_dims(lb,-1)  
    imgn = np.concatenate((img,img2),-1)
    return np.array(imgn,dtype=np.float32), np.array(lb,dtype=np.float32)

def data_generator(path,label, batch_size, step):
    import numpy as np
    import os
    trpath = []
    trlabel = []
    trpath = path.copy()
    trlabel = label.copy()
    i = 0
    while i <= step:
        if i == step:            
            i = 0            
        if i == 0:            
            rbatch_path = []
            rbatch_label = []            
            index = np.random.permutation(len(trpath))
            rrbatch_path = []
            rrbatch_label = []
            index = np.random.permutation(len(trpath))
            for ind in index:
                rrbatch_path.append(trpath[ind])
                rrbatch_label.append(trlabel[ind])
        batch_img = []
        batch_gt = []    
        for fi in range(batch_size):
            if (batch_size*i+fi) == len(rrbatch_path):
                break
            else:
                img, gt = process(rrbatch_path[batch_size*i+fi],rrbatch_label[batch_size*i+fi])
                batch_img.append(img)
                batch_gt.append(gt)
        i += 1
        yield np.array(batch_img), np.array(batch_gt)
        
def tdata_generator(tpath, tlabel, tbatch_size, step):
    import numpy as np
    import os    
    i = 0
    while i <= step:
        if i == step:
            i = 0                    
        batch_img = []
        batch_gt = []
        if tbatch_size == 1:
            img, gt = tprocess(tpath[tbatch_size*i],tlabel[tbatch_size*i])
            batch_img.append(img)
            batch_gt.append(gt)
        else:
            for fi in range(tbatch_size):
                img, gt = tprocess(tpath[tbatch_size*i+fi],tlabel[tbatch_size*i+fi])            
                batch_img.append(img)
                batch_gt.append(gt)
        i += 1
        yield np.array(batch_img), np.array(batch_gt)

In [ ]:
#===========================================================Model==============================================================================
from keras.optimizers import *
from keras.models import *
from keras import regularizers
from keras.utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from switchnorm import SwitchNormalization
from keras.layers import *
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

def dice_coef(y_true, y_pred):
    from keras import backend as K
    smooth = 1e-5
    intersection = K.sum(y_true * y_pred,axis=[1,2,3,4])
    return K.mean((2. * intersection + smooth) / (K.sum(y_true,axis=[1,2,3,4]) + K.sum(y_pred,axis=[1,2,3,4]) + smooth),axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.3
    smooth = 1e-5
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)
#https://blog.csdn.net/thunder_k/article/details/99202013

d = []
l = []
vd = []
vl = []
para_num = 2 #T1W+C and T2W images
inputs = Input((256,216,20,para_num))
conv01 = Conv3D(10,kernel_size=(3,3,1),strides=(1,1,1),padding='same',activation='relu')(inputs)
conv01 = SwitchNormalization()(conv01)
conv02 = Conv3D(10,kernel_size=(1,1,3),strides=(1,1,1),padding='same',activation='relu')(inputs)
conv02 = SwitchNormalization()(conv02)

conv11 = Conv3D(16,kernel_size=(3,3,1),strides=(1,1,1),padding='same',activation='relu')(conv01)
conv11 = SwitchNormalization()(conv11)
conv12 = Conv3D(16,kernel_size=(1,1,3),strides=(1,1,1),padding='same',activation='relu')(conv02)
conv12 = SwitchNormalization()(conv12)

conv21 = MaxPooling3D(pool_size=(2,2,1),strides=(2,2,1),padding='same')(conv11)
conv21 = Conv3D(32,kernel_size=(3,3,1),strides=(1,1,1),padding='same',activation='relu')(conv21)
conv21 = SwitchNormalization()(conv21)
conv22 = MaxPooling3D(pool_size=(2,2,1),strides=(2,2,1),padding='same')(conv12)
conv22 = Conv3D(32,kernel_size=(1,1,3),strides=(1,1,1),padding='same',activation='relu')(conv22)
conv22 = SwitchNormalization()(conv22)

conv31 = Conv3D(32,kernel_size=(3,3,1),strides=(1,1,1),padding='same',activation='relu')(conv21)
conv31 = SwitchNormalization()(conv31)
conv32 = Conv3D(32,kernel_size=(1,1,3),strides=(1,1,1),padding='same',activation='relu')(conv22)
conv32 = SwitchNormalization()(conv32)

convc = concatenate([MaxPooling3D(pool_size=(2,2,1),strides=(2,2,1),padding='same')(conv31),MaxPooling3D(pool_size=(2,2,1),strides=(2,2,1),padding='same')(conv32)],axis=-1)
convc = Conv3D(64,kernel_size=(3,3,3),strides=(1,1,1),padding='same',activation='relu')(convc)
convc = SwitchNormalization()(convc)

convc1 = Conv3D(32,kernel_size=(3,3,3),strides=(1,1,1),padding='same',activation='relu')(convc)
convc1 = SwitchNormalization()(convc1)

conv4 = Conv3DTranspose(16,kernel_size=(3,3,3),strides=(2,2,1),padding='same',activation='relu')(convc1)
conv4 = SwitchNormalization()(conv4)
conv4 = concatenate([Conv3D(32, kernel_size=(1,1,1),strides=(1,1,1),padding='same')(concatenate([conv31,conv32],axis=-1)),conv4],axis=-1)

conv5 = Conv3D(24,kernel_size=(3,3,3),strides=(1,1,1),padding='same',activation='relu')(conv4)
conv5 = SwitchNormalization()(conv5)

conv6 = Conv3DTranspose(12,kernel_size=(3,3,3),strides=(2,2,1),padding='same',activation='relu')(conv5)
conv6 = SwitchNormalization()(conv6)
conv6 = concatenate([Conv3D(16, kernel_size=(1,1,1),strides=(1,1,1),padding='same')(concatenate([conv11,conv12],axis=-1)),conv6],axis=-1)

conv7 = Conv3D(14,kernel_size=(3,3,3),strides=(1,1,1),padding='same',activation='relu')(conv6)
conv7 = SwitchNormalization()(conv7)

conv18 = Conv3D(1,kernel_size=(1,1,1),strides=(1,1,1),padding='same',activation='sigmoid')(conv7)

In [ ]:
#=============================================Training configuration===============================================================
model = Model(inputs, conv18)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

#====if have 2+ GPUS====
#parallel_model = multi_gpu_model(model, gpus=2)
# replace the 'model' below to 'parallel_model'
#=======================

model.compile(optimizer = Adam(lr=1e-3), loss = dice_coef_loss, metrics = [dice_coef])
modelcheckpoint = ModelCheckpoint(spath + 'model.h5',
                                     monitor='dice_coef',save_best_only=True,mode='max')
history = model.fit_generator(
    data_generator(path, label, batch_size, steps_per_epoch),
    epochs=epoch_num,
    steps_per_epoch=steps_per_epoch,
    validation_data=tdata_generator(tpath, tlabel, tbatch_size, steps_of_test),
    validation_steps = steps_of_test,
    callbacks=[modelcheckpoint])

d = history.history['dice_coef']
vd = history.history['val_dice_coef']
l = history.history['loss']
vl = history.history['val_loss']

model = Model(inputs, conv18)
model.compile(optimizer=SGD(lr=1e-4,momentum=0.9),loss = dice_coef_loss, metrics = [dice_coef])
model.load_weights(spath + 'model.h5')
modelcheckpoint = ModelCheckpoint((spath + 'k'+ str(kepoch_num) + 'model.h5'),
                                  monitor='dice_coef',save_best_only=True,mode='max')
mhistory = model.fit_generator(
    data_generator(path, label, batch_size, steps_per_epoch),
    epochs=kepoch_num,    
    steps_per_epoch=steps_per_epoch,
    validation_data=tdata_generator(tpath, tlabel, tbatch_size, steps_of_test),
    validation_steps = steps_of_test,
    callbacks=[modelcheckpoint])

d[len(d):len(d)] = mhistory.history['dice_coef']
vd[len(vd):len(vd)] = mhistory.history['val_dice_coef']
l[len(l):len(l)] = mhistory.history['loss']
vl[len(vl):len(vl)] = mhistory.history['val_loss']

import pandas as pd    
df = pd.DataFrame({'trdice':d,
                       'trloss':l,
                       'vdice':vd,
                       'vloss':vl,
                  })
writer = pd.ExcelWriter(spath + '/Dice&loss.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()

#====if parallel_model was used====
#p_model = load_model(spath + 'keras_M4.model',custom_objects={'tversky_loss': tversky_loss,'dice_coef':dice_coef})
#o_model = p_model.layers[-2]
#o_model.save(spath + 'k'+ str(kepoch_num) + 'keras_M4_single.model')
#==================================

print('done')